In [ ]:
### Part 1: embedding a trained generative AI model into a music app

In [ ]:
import urllib.request

In [ ]:
urllib.request.urlretrieve("https://play.forum.ircam.fr/rave-vst-api/get_model/darbouka_onnx", "darbouka_onnx.ts")

In [ ]:
from app.audio import create_audio_engine

In [ ]:
audio_engine = create_audio_engine("test-loop.wav", "darbouka_onnx.ts")

In [ ]:
audio_engine.start()

In [ ]:
audio_engine.loop.set()

In [ ]:
audio_engine.transform.set()

In [ ]:
audio_engine.set_latent_coordinates([0.1, 0.1, 0.5, 0.5])

In [ ]:
audio_engine.transform.clear()

In [ ]:
audio_engine.stop.set()

In [ ]:
audio_engine.stop.clear()